## Inheritance

**Inheritance provides**
- reusability
- extensibility
- avoids code duplication

In the last lecture, we saw how classes bring structure and readability to our code

Specifically, Parent class defines the interface and structure first.
Then, child classes implement it accordingly

Moving forward from our geometry example, here is the parent class

In [6]:
from abc import ABC, abstractmethod
from math import pi

# Parent Class
class Shape(ABC):
    @abstractmethod
    def draw(self):
        pass
    
    @abstractmethod
    def area(self):
        pass

Below we see how Child classes - Circle and Square implement the abstract methods as per their definitions

In [3]:
# Child Class
class Circle(Shape):
    def __init__(self,**kwargs):
        self.radius = kwargs['radius']
        
    def draw(self):
        return "Drawing a circle"
        
    @property
    def area(self):
        return pi * self.radius ** 2

class Square(Shape):
    def __init__(self,**kwargs):
        self.side = kwargs['side']
        
    def draw(self):
        return "Drawing a square"
        
    @property
    def area(self):
        return self.side ** 2

## Taking Inheritance Further

Suppose we want to create a class which has a combination of two geometries e.g.
- Circle with a hole
- Square with a hole

(Assume that a hole is always of circular shape)

One obvious approach is using Inheritance which can be read as
- CircleWithHole is a circle with a hole (is-A relation)
- SquareWithHole is a square with a hole (is-A relation)

In [4]:
# CircleWithHole -> is-A -> Circle
class CircleWithHole(Circle):
    def __init__(self,**kwargs):
        super().__init__(radius=kwargs['radius'])
        self.hole_radius = kwargs['hole_radius']
        
    def draw(self):
        return "Drawing a circle with hole"
        
    @property
    def area(self):
        return super().area - pi * self.hole_radius ** 2

In [7]:
# CLIENT CODE
geom_type = 'circleWithHole'
radius = 2
side = 2
hole_radius = 1

if geom_type == 'circle':
    geom = Circle(radius=radius)
elif geom_type == 'square':
    geom = Square(side=side)
elif geom_type == 'circleWithHole':
    geom = CircleWithHole(radius=radius,hole_radius=hole_radius)
else:
    ...
    
print(f'{geom.draw()} of area {geom.area}')

Drawing a circle with hole of area 9.42477796076938


# Problem with Inheritance

### 1. Class Hierarchy introduces Coupling
We can see how CircleWithHole sub-class has to know about its super class in init method
If we changed the definition of super class, it has to be changed in child class as well.
This is called coupling

To understand this, let's change the definition of `Circle` class to take another parameter `color` and let's also modify the corresponding `draw` method

In [10]:
class Circle(Shape):
    def __init__(self,**kwargs):
        self.radius = kwargs['radius']
        self.color = kwargs['color']
        
    def draw(self):
        return f"Drawing a {self.color} circle"
        
    @property
    def area(self):
        return pi * self.radius ** 2

To consume this change in super class `Circle`, we also need to change it in the sub-class `CircleWithHole`.

This coupling issue can become a big pain in production code

In [11]:
class CircleWithHole(Circle):
    def __init__(self,**kwargs):
        super().__init__(radius=kwargs['radius'],color=kwargs['color'])
        self.hole_radius = kwargs['hole_radius']
        
    def draw(self):
        return f"Drawing a {self.color} circle with hole"
        
    @property
    def area(self):
        return super().area - pi * self.hole_radius ** 2

In [14]:
# CLIENT CODE
geom_type = 'circleWithHole'
radius = 2
side = 2
hole_radius = 1
color = 'red'

if geom_type == 'circle':
    geom = Circle(radius=radius,color=color)
elif geom_type == 'circleWithHole':
    geom = CircleWithHole(radius=radius,color=color,hole_radius=hole_radius)
else:
    ...
    
print(f'{geom.draw()} of area {geom.area}')

Drawing a red circle with hole of area 9.42477796076938


### 2. Indeterminate explosion of child classes
Suppose we want to create more hieararchy of sub-classes e.g.
- SquareWithHole
- HexagonWithHole
- CircleWithHoleAndPadding
- etc.

We would need to implement each and every child classes which will involve lot of duplicate code.

In the example below, we implement `SquareWithHole` sub-class. 

Notice how the area method is completely same, still it needs to be implemented in each child class

In [8]:
# SquareWithHole -> is-A -> Square
class SquareWithHole(Square):
    def __init__(self,**kwargs):
        super().__init__(side=kwargs['side'])
        self.hole_radius = kwargs['hole_radius']
        
    def draw(self):
        return "Drawing a square with hole"
        
    @property
    def area(self):
        return super().area - pi * self.hole_radius ** 2